In [17]:
from IPython.utils import io
import multiprocessing
from matplotlib import pyplot
import numpy
import pygmo

# Sets random number generator
numpy.random.seed(1234)

# imports from the pyfoomb package
import pyfoomb
print(f'Current package version of {pyfoomb.__name__}: {pyfoomb.__version__}')

from pyfoomb import BioprocessModel
from pyfoomb import Caretaker
from pyfoomb import Visualization
from pyfoomb import TimeSeries
from pyfoomb import Helpers

Current package version of pyfoomb: 2.17.6


In [18]:
# Defines the model class
class YeastFedBatch(BioprocessModel):
    
    def rhs(self, y, t, sw):
       
    # Unpack state vector, alphabetically ordered
        
        A, B, G, E, N, V = y
        
        
        '''
        Variables name reference:
        A = Alpha
        B = Biomass
        G = Glucose
        E = Ethanol
        N = Nitrogen
        V = Volume
        
        '''
    
    # Unpacks the model parameters.
       
        k1 = self.model_parameters['k1']
        k2 = self.model_parameters['k2']
        k3 = self.model_parameters['k3']
        k4 = self.model_parameters['k4']
        k5 = self.model_parameters['k5']
        k6 = self.model_parameters['k6']
        u_Omax = self.model_parameters['u_Omax']
        u_Gmax = self.model_parameters['u_Gmax']
        u_Nmax = self.model_parameters['u_Nmax']
        k_G = self.model_parameters['k_G']
        k_l = self.model_parameters['k_l']
        k_N = self.model_parameters['k_N']
        k_A = self.model_parameters['k_A']
        k_lA = self.model_parameters['k_lA']
        k_lA2 = self.model_parameters['k_lA2']
        mu1_set = self.model_parameters['mu1_set']
        mu2_set = self.model_parameters['mu2_set']
        cSG = self.model_parameters['cSG']
        cSN = self.model_parameters['cSN']
        tF = self.model_parameters['tF']
        k_E = self.model_parameters['k_E']
        VL_max = self.model_parameters['VL_max']
        


        
        # For calculation of F, these three initial values are needed
        
        G0 = self.initial_values['G0']
        N0 = self.initial_values['N0']
        V0 = self.initial_values['V0']
        
        # Calculate the feeding profile, conditional to the corresponding events
        # There two variables needed; F1 and F2

        if sw[1] and not sw[0]:
            F1 = (G0*V0*mu1_set) / (cSG-G) * numpy.exp(mu1_set*(t-tF))
        else:
            F1 = 0.0
            
        if sw[1] and not sw[0]:
            F2 = (N0*V0*mu2_set) / (cSN-N) * numpy.exp(mu2_set*(t-tF))
        else:
            F2 = 0.0
        
        
        
        '''
The kinetic models associated with the global oxygen con-
sumption rO, the global glucose consumption rG and the global
nitrogen consumption rN
        '''
        r0 = u_Omax *(k1/E+k1)
        rg = u_Gmax * (G/(G+k_G))*(k_lA/((A*B)+k_lA))
        rn = u_Nmax * (N/(N+k_N))*(A*B/(A*B + k_A))*(k_lA2/((A*B)+k_lA2))
        
        # Rates caulculation
        
        r_1 = min(rg,r0)
        r_2 = max(0,rg-r0)
        r_3 = max (0,(r0-rg)*(E/E+k_E))
        r_4 = rn
        
        '''
        Hence, the mass balance equations for the different species in a
fed-batch bioreactor are given by the following equations
        '''
        dBdt = k1*r_1*B+k2*r_2*B+k3*r_3+k6*r_4*B-((F1 + F2)/V)*B
        dGdt = -r_1*B-r_2*B+(F1/V)*G0-((F1+F2)/V)*G
        dNdt = -r_4*B+(F2/V)*N0-((F1+F2)/V)*N
        dEdt = k2*r_2*B-r_3*B-((F1+F2)/V)*E
        dAdt = k5*r_2-r_4-A*(k1*r_1 + k2*r_2+k3*r_3+k6*r_4)
        dVdt = F1+F2
        
        return [dAdt,dBdt,dGdt,dEdt,dNdt,dVdt]
    
    def state_events(self, t, y, sw):
        
        A, B, G, E, N, V = y
        VL_max = self.model_parameters['VL_max']
        # The event is hit, when this expression evaluates to zero
        event_VL_max = V - VL_max
        
        tF = self.model_parameters['tF']
        # The event is hit, when this expression evaluates to zero
        event_tF = t - tF
        return [event_VL_max, event_tF]
    
    
    
model_parameters = {
        'k1': 0.5998,
        'k2': 0.0662,
        'k3': 0.9386, 
        'k4': 0.2452,  
        'k5': 0.2389, 
        'k6': 1.015,
        'u_Omax': 0.4445,
        'u_Gmax': 2.5364,  
        'u_Nmax': 1.1903,  
        'k_G': 0.1524, 
        'k_l': 3.1817, 
        'k_N': 2.9370,  
        'k_A': 9.0014,
        'k_lA':5.5981, 
        'k_lA2': 5.7737,
        'mu1_set': 0.15, 
        'mu2_set': 0.15, 
        'cSG' :  500.0,
        'cSN':   500.0,
        'tF':  200.0,
        'k_E' : 0.1,
        'VL_max': 20.0,
        
}   

initial_values = {
        'A0' : 0.0,
        'B0' : 0.1,
        'G0' : 0.1,
        'N0' : 0.1,
        'E0' : 0.0,
        'V0' : 1.0
}
        
caretaker = Caretaker(
    bioprocess_model_class=YeastFedBatch, 
    model_parameters=model_parameters, 
    initial_values=initial_values,
)
    

In [20]:
simulation = caretaker.simulate(t=20)
_ = Visualization.show_kinetic_data(simulation, ncols=2)

CVodeError occured with flag -9. CVodeError message was: 'The right-hand side function failed at the first call. At time 0.000000.'.


CVodeError: 'The right-hand side function failed at the first call. At time 0.000000.'